In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pymrmr

In [ ]:
seed = 42

# Import libraries
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split

import random
random.seed(seed)

import matplotlib.pyplot as plt
import cv2
#from google.colab.patches import cv2_imshow
import numpy as np
import os
import glob
from PIL import Image
from imgaug import augmenters as iaa
from skimage.transform import resize

import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


## Import Ste

In [ ]:
X_train=np.load("/kaggle/input/immagini-nuove/images_array.npy")
Y_train=np.load("/kaggle/input/immagini-nuove/labels_array.npy")

In [ ]:
print(np.unique(Y_train, return_counts = True))

## Import Ale

In [ ]:
X_train=np.load("/kaggle/input/dataset/archive-2/images_array.npy")
Y_train=np.load("/kaggle/input/dataset/archive-2/labels_array.npy")

In [ ]:
# Dividi i dati in set di addestramento, validazione e test
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=32)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=32) 

# Assicurati che le dimensioni siano corrette
print("Dimensioni set di addestramento:", x_train.shape, y_train.shape)
print("Dimensioni set di validazione:", x_val.shape, y_val.shape)
print("Dimensioni set di test:", x_test.shape, y_test.shape)


x_train=x_train.astype('float32')/255.
x_val=x_val.astype('float32')/255.
x_test=x_test.astype('float32')/255.

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
print(np.unique(y_train, return_counts = True))
print(np.unique(y_val, return_counts = True))
print(np.unique(y_test, return_counts = True))


In [ ]:
from tensorflow.keras.optimizers import AdamW

class PrintLearningRate(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = float(self.model.optimizer.learning_rate.numpy())  # Ottenere il valore numerico del tasso di apprendimento
        print(f'Epoch {epoch+1}: Learning Rate = {lr}')

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitora la perdita sulla tua valutazione
    patience=20,          # Numero di epoche senza miglioramenti prima di fermare l'addestramento
    restore_best_weights=True  # Ripristina i pesi del modello alla migliore iterazione
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitora la perdita sulla tua valutazione
    factor=0.1,          # Fattore di riduzione del tasso di apprendimento (riduce il tasso di apprendimento di un fattore di 0.2)
    patience=10,          # Numero di epoche senza miglioramenti prima di ridurre il tasso di apprendimento
    min_lr=1e-10         # Limite inferiore del tasso di apprendimento
)

learning_rate = 1e-3


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D,BatchNormalization
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1


### Modello ALE
def encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    x = Flatten()(x)
    encoded = Dense(64, activation='relu', kernel_regularizer=l1(10e-8))(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    encoder_model.summary()
    return encoder_model

def decoder(output_shape):
    inputs = Input(shape=output_shape)
    x = Dense(2*2*256, activation='relu')(inputs)
    x = Reshape((2, 2, 256))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    #x = UpSampling2D((2, 2))(x)
    #x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    #x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    #x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    decoder_model = Model(inputs, decoded, name='decoder')
    decoder_model.summary()
    return decoder_model



input_shape = (64, 64, 1)
encoder_model = encoder(input_shape)
decoder_model = decoder(encoder_model.output_shape[1:])

inputs = Input(shape=input_shape)
encoded = encoder_model(inputs)
decoded = decoder_model(encoded)
autoencoder_model = Model(inputs, decoded, name='autoencoder')


# Compilazione dell'autoencoder
autoencoder_model.compile(optimizer=AdamW(learning_rate=learning_rate), loss='binary_crossentropy')

# Visualizzazione dell'architettura dell'autoencoder
autoencoder_model.summary()



In [ ]:
# Addestramento dell'autoencoder
autoencoder_model.fit(x_train, x_train,
                      epochs=1000,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(x_val, x_val),
                      callbacks=[early_stopping, reduce_lr])

#encoder_model.save('encoder_model16.h5')

In [ ]:
import matplotlib.pyplot as plt

# Numero di immagini da visualizzare
num_images = 10

reconstructed_images = autoencoder_model.predict(x_test)


# Visualizzazione delle immagini originali e ricostruite
plt.figure(figsize=(15, 6))
for i in range(num_images):
    # Immagine originale
    plt.subplot(2, num_images, i + 1)
    plt.imshow(x_test[i], cmap='gray')
    plt.title('Originale')
    plt.axis('off')

    # Immagine ricostruita
    plt.subplot(2, num_images, i + num_images + 1)
    plt.imshow(reconstructed_images[i], cmap='gray')
    plt.title('Ricostruita')
    plt.axis('off')

plt.tight_layout()
plt.show()


# classificatori

## Ottenimento features


In [ ]:
import numpy as np
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

#encoder_model = load_model('encoder_model16.h5')
#encoder_model = load_model('encoder_model32.h5')
#encoder_model = load_model('encoder_model64.h5')

#encoder_model = load_model('/kaggle/input/modelli/encoder_model16.h5')
encoder_model = load_model('/kaggle/input/modelli/encoder_model32.h5')
#encoder_model = load_model('/kaggle/input/modelli/encoder_model64.h5')

# Usa l'encoder per ottenere le feature da ciascun set
X_train_encoded = encoder_model.predict(x_train)
X_val_encoded = encoder_model.predict(x_val)
X_test_encoded = encoder_model.predict(x_test)

# Verifica la forma delle feature estratte
print(f"Shape of encoded train features: {X_train_encoded.shape}")
print(f"Shape of encoded val features: {X_val_encoded.shape}")
print(f"Shape of encoded test features: {X_test_encoded.shape}")

## Feature Selector

In [ ]:
import pymrmr
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA

def feature_selection(X_train, X_val, X_test, y_train, num_features_mrmr=None, alpha_lasso=None, num_features_rf=None, method=None):
    if method == "MRMR":
        # Feature selection using MRMR
        df_train = pd.DataFrame(X_train)
        df_train.columns = df_train.columns.astype(str)
        selected_features_mrmr = pymrmr.mRMR(df_train, 'MIQ', num_features_mrmr)
        selected_features_mrmr_int = [int(feature) for feature in selected_features_mrmr]
        
        X_train_selected = X_train[:, selected_features_mrmr_int]
        X_val_selected = X_val[:, selected_features_mrmr_int]
        X_test_selected = X_test[:, selected_features_mrmr_int]
        
        print("Le caratteristiche selezionate tramite MRMR sono:", selected_features_mrmr)
        
    elif method == "LASSO":
        # Feature selection using LASSO
        lasso = Lasso(alpha=alpha_lasso)
        lasso.fit(X_train, y_train)
        coefficients = lasso.coef_
        selected_features_lasso = np.where(coefficients != 0)[0]
        
        X_train_selected = X_train[:, selected_features_lasso]
        X_val_selected = X_val[:, selected_features_lasso]
        X_test_selected = X_test[:, selected_features_lasso]
        
        print("Le caratteristiche selezionate tramite LASSO sono:", selected_features_lasso)
        
    elif method == "RF":
        # Feature selection using Random Forest
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        importances = rf.feature_importances_
        indices = np.argsort(importances)[::-1]
        selected_features_rf = indices[:num_features_rf]
        
        X_train_selected = X_train[:, selected_features_rf]
        X_val_selected = X_val[:, selected_features_rf]
        X_test_selected = X_test[:, selected_features_rf]
        
        print("Le caratteristiche selezionate tramite Random Forest sono:", selected_features_rf)
        
    else:
        raise ValueError("Invalid method specified. Choose from 'MRMR', 'LASSO', or 'RF'.")
    
    return X_train_selected, X_val_selected, X_test_selected



### MLP

In [ ]:
def train_and_evaluate_mlp(X_train, y_train, X_val, y_val, X_test, y_test, hidden_layers=(128, 64, 32), max_iter=500, random_state=42, batch_size=64):
    # Inizializza il classificatore MLP
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layers, max_iter=max_iter, random_state=random_state, 
                        verbose=True, batch_size=batch_size, early_stopping=True, learning_rate="adaptive")

    # Addestra il classificatore sulle feature selezionate dal set di addestramento
    mlp.fit(X_train, y_train)

    # Fai predizioni sui dati di validazione e test
    y_val_pred = mlp.predict(X_val)
    y_test_pred = mlp.predict(X_test)

    # Calcola l'accuratezza sui set di validazione e test
    val_accuracy = accuracy_score(y_val, y_val_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Calcola precisione e richiamo sul set di validation
    precision_val = precision_score(y_val, y_val_pred)
    recall_val = recall_score(y_val, y_val_pred)
    
    # Calcola precisione e richiamo sul set di test
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)

    # Calcola la matrice di confusione sul set di validation
    conf_matrix_val = confusion_matrix(y_val, y_val_pred)
    
    # Calcola la matrice di confusione sul set di test
    conf_matrix = confusion_matrix(y_test, y_test_pred)

    # Stampa i risultati
    print()
    print(f"Validation Accuracy: {val_accuracy}")
    print(f"Test Accuracy: {test_accuracy}")
    print()
    print("Precision VALIDATION:", precision_val)
    print("Recall VALIDATION:", recall_val)
    print()
    print("Precision TEST:", precision)
    print("Recall TEST:", recall)
    print()
    print('Confusion Matrix VALIDATION:')
    print(conf_matrix_val)
    print()
    print('Confusion Matrix TEST:')
    print(conf_matrix)

    return {
        "val_accuracy": val_accuracy,
        "test_accuracy": test_accuracy,
        "precision_val": precision_val,
        "recall_val": recall_val,
        "precision_test": precision,
        "recall_test": recall,
        "confusion_matrix_val": conf_matrix_val,
        "confusion_matrix_test": conf_matrix
    }

### Vanilla

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.optimizers import AdamW, Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def build_and_train_nn(X_train, y_train, X_val, y_val, X_test, y_test, input_dim, learning_rate=1e-3, batch_size=64, epochs=1000):
    # Definisci il modello
    model = Sequential()

    model.add(Input(shape=(input_dim,)))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Livello di output
    model.add(Dense(1, activation='sigmoid'))

    # Compila il modello
    model.compile(optimizer=AdamW(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    # Mostra il sommario del modello
    model.summary()

    # Callbacks
    early_stopping_class = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    reduce_lr_class = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10)

    
    # Addestra il modello utilizzando i dati di training e di validazione
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[early_stopping_class, reduce_lr_class])

    # Fai predizioni sui dati di test e val
    y_val_pred_proba = model.predict(X_val)
    y_val_pred = (y_val_pred_proba > 0.5).astype(int)
    
    y_test_pred_proba = model.predict(X_test)
    y_test_pred = (y_test_pred_proba > 0.5).astype(int)

    val_accuracy = accuracy_score(y_val, y_val_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print()
    print(f"Validation Accuracy: {val_accuracy}")
    print(f"Test Accuracy: {test_accuracy}")
    
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    print()
    print(f"Precision Validation: {precision:.4f}")
    print(f"Recall Validation: {recall:.4f}")
    print()    
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    print()
    print(f"Precision Test: {precision:.4f}")
    print(f"Recall Test: {recall:.4f}")
    print()
    print('Confusion Matrix VALIDATION:')
    print(confusion_matrix(y_val, y_val_pred))
    print()
    print('Confusion Matrix TEST:')
    print(confusion_matrix(y_test, y_test_pred))

    return model, history


## TESTS

### Creazione Variabili

In [ ]:
X_train_selected_mrmr_32, X_val_selected_mrmr_32, X_test_selected_mrmr_32 = feature_selection(X_train_encoded, X_val_encoded, X_test_encoded, y_train, num_features_mrmr=24, alpha_lasso=0.001, num_features_rf=24, method="MRMR")


In [ ]:
X_train_selected_mrmr_48, X_val_selected_mrmr_48, X_test_selected_mrmr_48 = feature_selection(X_train_encoded, X_val_encoded, X_test_encoded, y_train, num_features_mrmr=12, alpha_lasso=0.001, num_features_rf=24, method="MRMR")


In [ ]:
X_train_selected_rf_32, X_val_selected_rf_32, X_test_selected_rf_32 = feature_selection(X_train_encoded, X_val_encoded, X_test_encoded, y_train, num_features_mrmr=12, alpha_lasso=0.001, num_features_rf=24, method="RF")


In [ ]:
X_train_selected_rf_48, X_val_selected_rf_48, X_test_selected_rf_48 = feature_selection(X_train_encoded, X_val_encoded, X_test_encoded, y_train, num_features_mrmr=12, alpha_lasso=0.001, num_features_rf=12, method="RF")


In [ ]:
X_train_selected_lasso_34, X_val_selected_lasso_34, X_test_selected_lasso_34 = feature_selection(X_train_encoded, X_val_encoded, X_test_encoded, y_train, num_features_mrmr=12, alpha_lasso=0.0001, num_features_rf=24, method="LASSO")

## Risultati Modelli

### Vanilla

In [ ]:
## RANDOM FOREST 32
model, history = build_and_train_nn(X_train_selected_rf_32, y_train, X_val_selected_rf_32, y_val, X_test_selected_rf_32, y_test, X_train_selected_rf_32.shape[1])


In [ ]:
model, history = build_and_train_nn(X_train_selected_rf_48, y_train, X_val_selected_rf_48, y_val, X_test_selected_rf_48, y_test, X_train_selected_rf_48.shape[1])


In [ ]:
## MRMR 16
model, history = build_and_train_nn(X_train_selected_mrmr_32, y_train, X_val_selected_mrmr_32, y_val, X_test_selected_mrmr_32, y_test, X_train_selected_mrmr_32.shape[1])


In [ ]:
## MRMR 16
model, history = build_and_train_nn(X_train_selected_mrmr_48, y_train, X_val_selected_mrmr_48, y_val, X_test_selected_mrmr_48, y_test, X_train_selected_mrmr_48.shape[1])


In [ ]:
model, history = build_and_train_nn(X_train_selected_lasso_34, y_train, X_val_selected_lasso_34, y_val, X_test_selected_lasso_34, y_test, X_train_selected_lasso_34.shape[1])


### MLP

In [ ]:
## RANDOM FOREST 24
result = train_and_evaluate_mlp(X_train_selected_rf_32, y_train, X_val_selected_rf_32, y_val, X_test_selected_rf_32, y_test)

In [ ]:
result = train_and_evaluate_mlp(X_train_selected_rf_48, y_train, X_val_selected_rf_48, y_val, X_test_selected_rf_48, y_test)

In [ ]:
## MRMR 24
result = train_and_evaluate_mlp(X_train_selected_mrmr_32, y_train, X_val_selected_mrmr_32, y_val, X_test_selected_mrmr_32, y_test)

In [ ]:
result = train_and_evaluate_mlp(X_train_selected_mrmr_48, y_train, X_val_selected_mrmr_48, y_val, X_test_selected_mrmr_48, y_test)

In [ ]:
result = train_and_evaluate_mlp(X_train_selected_lasso_34, y_train, X_val_selected_lasso_34, y_val, X_test_selected_lasso_34, y_test)

### Without Selection

In [ ]:
model, history = build_and_train_nn(X_train_encoded, y_train, X_val_encoded, y_val, X_test_encoded, y_test, X_train_encoded.shape[1])


In [ ]:
result = train_and_evaluate_mlp(X_train_encoded, y_train, X_val_encoded, y_val, X_test_encoded, y_test)


DA FARE:

- Finire mrmr 24 per vanilla e mlp
- Testare MID per MRMR
- Testare batch 64

- Eseguire codice su DENSE 64
- Eseguire codice su DENSE 16